In [ ]:
file_path="Data/ICICIBANK_1min_load.csv"

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv(file_path)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
print(f"Rows before: {len(df)}")

In [ ]:
df = df[~df.index.duplicated(keep='first')]

In [ ]:
print(f"Rows after: {len(df)}")

In [ ]:
required_cols = ['open', 'high', 'low', 'close', 'volume']
df.dropna(subset=required_cols, inplace=True)

In [ ]:
print(f"Rows after dropping NaNs: {len(df)}")

In [ ]:
perfect_index = pd.date_range(start="2026-01-09 09:30:00", 
                              end="2026-01-09 16:00:00", 
                              freq='1min')

In [ ]:
df_continuous = df.reindex(perfect_index)

In [ ]:
gaps_found = df_continuous['close'].isna().sum()
print(f"Missing minutes found: {gaps_found}")

In [ ]:
df_continuous[['open', 'high', 'low', 'close']] = df_continuous[['open', 'high', 'low', 'close']].ffill()
df_continuous['volume'] = df_continuous['volume'].fillna(0)

In [ ]:
df_continuous.bfill(inplace=True)

In [ ]:
print(df.columns)|

In [ ]:
print(df.columns)

In [ ]:
all_days = df.index.normalize().unique()

In [ ]:
df['date'] = pd.to_datetime(df['Date'], errors='coerce')

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

In [ ]:
df.set_index('timestamp', inplace=True)

In [ ]:
print(df.index)

In [ ]:
df

In [ ]:
all_days = df.index.normalize().unique()

In [ ]:
master_minutes = []
for day in all_days:
    day_range = pd.date_range(
        start=day.replace(hour=9, minute=30), 
        end=day.replace(hour=16, minute=0), 
        freq='1min'
    )
    master_minutes.extend(day_range)

In [ ]:
master_index = pd.DatetimeIndex(master_minutes)

In [ ]:
df_continuous = df.reindex(master_index)

In [ ]:
df_continuous[['open', 'high', 'low', 'close']] = df_continuous[['open', 'high', 'low', 'close']].ffill()

In [ ]:
df_continuous['volume'] = df_continuous['volume'].fillna(0)

In [ ]:
df_continuous = df_continuous.bfill().infer_objects(copy=False)

In [ ]:
print(f"Total rows in continuous data: {len(df_continuous)}")
print(f"Total gaps filled: {len(df_continuous) - len(df)}")

In [ ]:
df

In [ ]:
print(df_continuous[['open', 'high', 'low', 'close']].isnull().sum())

In [ ]:
print(df_continuous.tail(10))

In [ ]:
print(df_continuous.sample(10))

In [ ]:
df_continuous.to_csv("Data/NVDA_Cleaned_Final.csv", 
                     index=True, 
                     date_format='%Y-%m-%d %H:%M:%S',
                     float_format='%.4f')